# Seleção dos hyperparâmetros 

Hyperparâmetros são os parâmetros que não são aprendidos no processo de treinamento da rede neural. São parâmetros tanto do modelo, como número e tipo de camadas, 
número de neurônios ou canais em cada camada, como parâmetros do processo de otimização, tais como tipo do otimizador, taxa de aprendizado (learning rate), tamanho
do mini-batch, entre outros.

Uma opção é disparar um processo automático de busca no espaço de hyperparâmetros para buscar o melhor índice de validação cruzada. 

Normalmente a busca neste espaço de hyperparâmetros consiste de:
- o modelo da rede;
- o espaço de hyperparâmetros;
- o método de busca e amostragem neste espaço;
- o esquema de validação cruzada; e
- uma função alvo (*score function*)



<img src='../figures/model_selection.png', width=600></img>

## Validação cruzada

<img src='../figures/cross_validation.png', width=600></img>

## Objetivos desse experimento

Este experimento utiliza como:
- **modelo da rede:** fixa - Fully connected, já usada anteriormente
- **espaço de hyperparâmetros:** variando learning rate e decay
- **método de busca:** RandomizedSearch, onde número de iterações é especificado
- **validação cruzada:** n. de folds especificado, opção se usar ou não dados de teste
- **função alvo:** loss

## Importação dos pacotes tradicionais

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os, sys
import time
import numpy as np

import scipy.stats as st
import numpy.random as nr

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import torchvision as tv
import lib.pytorch_trainer as ptt

use_gpu = torch.cuda.is_available()
print('GPU available:', use_gpu)

np.set_printoptions(precision=3)

GPU available: True


## Carregamento dos dados - MNIST

In [2]:
train_ds = tv.datasets.MNIST('/data/datasets/MNIST/', train=True, 
                             transform=tv.transforms.ToTensor())
valid_ds = tv.datasets.MNIST('/data/datasets/MNIST/', train=False, 
                             transform=tv.transforms.ToTensor())
n_train = len(train_ds)
n_valid = len(valid_ds)
print('Número de amostras no dataset (treino):', n_train)
print('Número de amostras no dataset (teste ):', n_valid)

Número de amostras no dataset (treino): 60000
Número de amostras no dataset (teste ): 10000


### Reduzindo o tamanho do dataset apenas para acelerar e testar

In [3]:
if True:
    fator_reduc = 0.02
    n_train = int(fator_reduc * n_train)
    n_valid = int(fator_reduc * n_valid)
    train_ds.train_data   = train_ds.train_data[:n_train]
    train_ds.train_labels = train_ds.train_labels[:n_train]
    valid_ds.test_data   = valid_ds.test_data[:n_valid]
    valid_ds.test_labels = valid_ds.test_labels[:n_valid]

In [4]:
print('Número de amostras no dataset (treino):', len(train_ds))
print('Número de amostras no dataset (teste ):', len(valid_ds))

Número de amostras no dataset (treino): 1200
Número de amostras no dataset (teste ): 200


### Sampler

In [5]:
from torch.utils.data.sampler import Sampler

class MySampler(Sampler):
    def __init__(self, indexes):
        self.indexes = indexes

    def __iter__(self):
        return iter(self.indexes[torch.randperm(len(self.indexes)).long()])

    def __len__(self):
        return len(self.indexes)


## Definição de rede

In [6]:
class Model(nn.Module):
    def __init__(self, n=50):
        super().__init__()
        self.fc1 = nn.Linear(28 * 28, n)
        self.at1 = nn.ReLU()
        self.fc2 = nn.Linear(n, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.fc1(x)
        x = self.at1(x)
        x = self.fc2(x)
        return x

## Definição do RandomizedSearch

In [7]:
param_distributions = {
    'model_class':        None,
    'loss_class':         [nn.CrossEntropyLoss],
    'optim_class':        [optim.Adam], 
    'optim_lr':           st.uniform(0.0001, 0.005),
    'optim_weight_decay': st.uniform(0.0, 0.01),
    'optim_momentum':     [0.9],
    'batch_size':         [20],
    'num_epochs':         [10],
}

class MyRandomizedSearch(object):
    def __init__(self, params=param_distributions, num_splits=3, num_iter=10):
        if params['model_class'] is None:
            raise Exception('')
        self.parameters = params
        self.n_splits = num_splits
        self.n_iteractions = num_iter
        
    def fit(self, dataset, verbose=False):
        self.parameter_sets = []
        self.scores = []
        
        for it in range(self.n_iteractions):
            print('Iteraction {:2d}:'.format(it), end=' ')
            t0 = time.time()
            p = self.sample_parameter_space()
            self.parameter_sets.append(p)
            split_scores = []
            trainer = self.make_trainer(p, verbose=verbose)
            for train_dloader, valid_dloader in self.gen_data_loaders(dataset, 
                                                                      self.n_splits, 
                                                                      p['batch_size']):
                trainer.fit_loader(p['num_epochs'], train_dloader)
                metrics = trainer.evaluate_loader(valid_dloader, verbose=0)
                score = metrics['losses']
                split_scores.append(score)
                print('{:.5f}'.format(score), end=' ')
            self.scores.append(split_scores)
            print('[{:.1f} s]'.format(time.time() - t0))
            
        self.show_results()
            
    def sample_parameter_space(self):
        pars = dict()
        for p, v in self.parameters.items():
            if type(v) == list:
                pars[p] = v[0] if len(v) == 1 else nr.randint(0, len(v))
            elif getattr(v, 'rvs', None) is not None:
                pars[p] = v.rvs()
            else:
                raise Exception('Unknown par dist type')
        return pars
    
    def gen_data_loaders(self, dataset, n_splits, batch_size):
        n_total = len(dataset)
        split_size = n_total // n_splits
        indices = torch.arange(n_total)
        split = torch.LongTensor([n_splits - 1 for _ in indices])
        split[:n_splits * split_size] = torch.arange(n_splits * split_size).long() / split_size
        for i in range(n_splits):
            vii = indices[split == i].long()
            tii = indices[split != i].long()
            train_dloader = DataLoader(dataset, batch_size=batch_size, sampler=MySampler(tii))
            valid_dloader = DataLoader(dataset, batch_size=batch_size, sampler=MySampler(vii))
            yield train_dloader, valid_dloader
    
    def make_trainer(self, p, verbose):
        if verbose > 0:
            callbacks = [ptt.PrintCallback()]
        else:
            callbacks = None            
        self.model = p['model_class']()
        loss_fn = self.make_criterion(p)
        optimizer = self.make_optimizer(p)
        trainer = ptt.DeepNetTrainer(model=self.model, criterion=loss_fn, 
                                     optimizer=optimizer, callbacks=callbacks)
        return trainer
        
    def make_optimizer(self, p):
        if p['optim_class'] == 'Adam':
            optimz = optim.Adam(self.model.parameters(), lr=p['optim_lr'], 
                                weight_decay=p['optim_weight_decay'])
        elif p['optim_class'] == 'SGD':
            optimz = optim.SGD(self.model.parameters(), lr=p['optim_lr'], 
                               momentum=p['optim_momentum'], 
                               weight_decay=p['optim_weight_decay'], nesterov=True)
        elif p['optim_class'] == 'RMSprop':
            optimz = optim.RMSprop(self.model.parameters(), lr=p['optim_lr'], 
                                   weight_decay=p['optim_weight_decay'])
        else:
            raise Exception("A ser implementado...")
        return optimz
    
    def make_criterion(self, p):
        if p['loss_class'] == 'CrossEntropyLoss':
            loss_fn = nn.CrossEntropyLoss()
        elif p['loss_class'] == 'MSELoss':
            loss_fn = nn.MSELoss()
        else:
            raise Exception("A ser implementado...")
        return loss_fn
    
    def show_results(self):
        self.mean_scores = torch.FloatTensor(rs.scores).mean(1)
        self.best_loss, self.best_index = [x[0] for x in self.mean_scores.min(0)]
        self.best_trainer = self.make_trainer(rs.parameter_sets[self.best_index], verbose=0)
        print('\nBest parameter set from iteraction {} with loss = {:.5f}:'.format(self.best_index, self.best_loss))
        for p, v in self.parameter_sets[self.best_index].items():
            print('    {:20s}: {}'.format(p, v))
        

## Efetuando a busca

In [8]:
params = {
    'model_class':        [Model],
    'loss_class':         ['CrossEntropyLoss'],
    'optim_class':        ['Adam'], 
    'optim_lr':           st.uniform(0.0001, 0.005),
    'optim_weight_decay': st.uniform(0.0, 0.01),
    'optim_momentum':     [0.9],
    'weight_decay':       [0.0],
    'batch_size':         [20],
    'num_epochs':         [50],
}

rs = MyRandomizedSearch(params, num_splits=3, num_iter=10)
rs.fit(train_ds)


Iteraction  0: 0.48420 0.47077 0.46774 [37.3 s]
Iteraction  1: 0.41834 0.47786 0.53363 [33.3 s]
Iteraction  2: 0.50510 0.36839 0.31426 [33.3 s]
Iteraction  3: 0.48735 0.49378 0.51934 [33.5 s]
Iteraction  4: 0.46025 0.42248 0.37672 [33.6 s]
Iteraction  5: 0.43813 0.43806 0.41564 [33.4 s]
Iteraction  6: 0.41738 0.49789 0.50616 [33.2 s]
Iteraction  7: 0.43290 0.50205 0.49366 [33.2 s]
Iteraction  8: 0.44634 0.39853 0.39854 [33.3 s]
Iteraction  9: 0.43594 0.49803 0.50154 [33.3 s]

Best parameter set from iteraction 2 with loss = 0.39592:
    model_class         : <class '__main__.Model'>
    loss_class          : CrossEntropyLoss
    optim_class         : Adam
    optim_lr            : 0.0019526153512656936
    optim_weight_decay  : 0.0004022120167432541
    optim_momentum      : 0.9
    weight_decay        : 0.0
    batch_size          : 20
    num_epochs          : 50


# Exercícios

1. Adicione um parâmetro adicional na escolha do otimizador: 'SGV' e o 'Adam'.
2. Adicione agora um parâmetro na rede, por exemplo trocar a função de ativação de 'ReLU' para 'Sigmoid'.
3. Adicione um parâmetro a sua escolha.